# Exploring CAFE-f6 bias correction

In [1]:
import glob

import dask

import cftime

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

dask.config.set(**{"array.slicing.split_large_chunks": False})

In [2]:
from src.utils import utils

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [4]:
xr.set_options(keep_attrs=True)

# Cluster

In [5]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

walltime = "04:00:00"
cores = 48
memory = "192GB"
cluster = PBSCluster(
    processes=1,
    walltime=str(walltime),
    cores=cores,
    memory=str(memory),
    job_extra=[
        "-l ncpus=" + str(cores),
        "-l mem=" + str(memory),
        "-P xv83",
        "-l jobfs=100GB",
        "-l storage=gdata/xv83",
    ],
    local_directory="$PBS_JOBFS",
    # env_extra=['export MALLOC_TRIM_THRESHOLD_="0"'],
    header_skip=["select"],
)

/g/data/xv83/users/ds0092/software/miniconda3/envs/cafe-f6_analysis/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [6]:
cluster.scale(jobs=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.24.71:8787/status,
Dashboard: http://10.6.24.71:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.24.71:43139,Workers: 0
Dashboard: http://10.6.24.71:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Prepare some data - CAFE hindcasts
Use only the November forecasts, due to issue with changing executable for May forecasts

In [7]:
def preprocess_CAFEf6_monthly_to_annual(ds):
    """Preprocessing steps for CAFE-f6 data"""

    ds = utils.round_to_start_of_month(ds, dim="time")
    ds = utils.convert_time_to_lead(ds)

    ds = utils.coarsen_monthly_to_annual(ds, dim="lead")

    # Truncate lats so that forecasts run on different systems can be stacked
    if "lat" in ds.dims:
        ds = utils.truncate_latitudes(ds)

    return ds

In [8]:
def global_mean(ds, area, dim=["lat", "lon"]):
    return ds.weighted(area).mean(dim)

### Atmospheric variables

In [9]:
data_dir = "/g/data/xv83/dcfp/CAFE-f6"
realm = "atmos_isobaric_month.zarr.zip"
variable = {"t_ref": 1, "precip": 86400}  # Including scale factor

files = sorted(glob.glob(f"{data_dir}/c5-d60-pX-f6-????11??/{realm}"))

f6_hcst_atmos = xr.open_mfdataset(
    files,
    compat="override",
    preprocess=preprocess_CAFEf6_monthly_to_annual,
    engine="zarr",
    coords="minimal",
    parallel=True,
)[variable.keys()]

# Rescale variables
for k, v in variable.items():
    f6_hcst_atmos[k] = f6_hcst_atmos[k] * v

cafe_atmos_area = f6_hcst_atmos["area"]

# Focus on global mean for now
f5_2020_atmos = f6_hcst_atmos.sel(init="2020").isel(ensemble=range(10))
f6_hcst = global_mean(f6_hcst_atmos.mean("ensemble"), cafe_atmos_area).compute()

In [10]:
data_dir = "/g/data/xv83/dcfp/CAFE-f5/NOV/"
realm = "atmos_isobaric_month.zarr.zip"
variable = {"t_ref": 1, "precip": 86400}  # Including scale factor

f5_hcst_atmos = xr.open_dataset(f"{data_dir}{realm}", engine="zarr", chunks={})[
    variable.keys()
]
f5_hcst_atmos = f5_hcst_atmos.rename({"init_date": "init", "lead_time": "lead"})

# Rescale variables
for k, v in variable.items():
    f5_hcst_atmos[k] = f5_hcst_atmos[k] * v

f5_hcst_atmos = utils.coarsen_monthly_to_annual(f5_hcst_atmos, dim="lead")
f5_hcst_atmos = utils.round_to_start_of_month(f5_hcst_atmos, dim=["init", "time"])
f5_hcst_atmos = xr.concat((f5_hcst_atmos, f5_2020_atmos), dim="init")

# Focus on global mean for now
f5_hcst = global_mean(f5_hcst_atmos, cafe_atmos_area).mean("ensemble").compute()

### Ocean variables

In [ ]:
data_dir = "/g/data/xv83/dcfp/CAFE-f6"
realm = "ocean_month.zarr.zip"
variable = {"sst": 1}  # Including scale factor

files = sorted(glob.glob(f"{data_dir}/c5-d60-pX-f6-????11??/{realm}"))

f6_hcst_ocean = xr.open_mfdataset(
    files,
    compat="override",
    preprocess=preprocess_CAFEf6_monthly_to_annual,
    engine="zarr",
    coords="minimal",
    parallel=True,
)[variable.keys()]

# Rescale variables
for k, v in variable.items():
    f6_hcst_ocean[k] = f6_hcst_ocean[k] * v

cafe_ocean_area = f6_hcst_ocean["area_t"]

# Focus on global mean for now
f5_2020_ocean = f6_hcst_ocean.sel(init="2020").isel(ensemble=range(10))
f6_hcst = xr.merge(
    [
        f6_hcst,
        global_mean(
            f6_hcst_ocean.mean("ensemble"), cafe_ocean_area, ["xt_ocean", "yt_ocean"]
        ).compute(),
    ]
)

In [ ]:
data_dir = "/g/data/xv83/dcfp/CAFE-f5/NOV/"
realm = "ocean_month.zarr.zip"
variable = {"sst": 1}  # Including scale factor

f5_hcst_ocean = xr.open_dataset(f"{data_dir}{realm}", engine="zarr", chunks={})[
    variable.keys()
]
f5_hcst_ocean = f5_hcst_ocean.rename({"init_date": "init", "lead_time": "lead"})

# Rescale variables
for k, v in variable.items():
    f5_hcst_ocean[k] = f5_hcst_ocean[k] * v

f5_hcst_ocean = utils.coarsen_monthly_to_annual(f5_hcst_ocean, dim="lead")
f5_hcst_ocean = utils.round_to_start_of_month(f5_hcst_ocean, dim=["init", "time"])
f5_hcst_ocean = xr.concat((f5_hcst_ocean, f5_2020_ocean), dim="init")

# Focus on global mean for now
f5_hcst = xr.merge(
    [
        f5_hcst,
        global_mean(f5_hcst_ocean, cafe_ocean_area, ["xt_ocean", "yt_ocean"])
        .mean("ensemble")
        .compute(),
    ]
)

# Prepare some data - CAFE60

### Atmospheric variables

In [ ]:
data_dir = "/g/data/xv83/dcfp/CAFE60v1/"
realm = "atmos_isobaric_month.zarr.zip"
variable = {"t_ref": 1, "precip": 86400}

cafe_60_atmos = xr.open_dataset(f"{data_dir}{realm}", engine="zarr", chunks={})[
    variable.keys()
]
cafe_60_atmos = utils.truncate_latitudes(cafe_60_atmos)

# Rescale variables
for k, v in variable.items():
    cafe_60_atmos[k] = cafe_60_atmos[k] * v

cafe_60_atmos = utils.coarsen_monthly_to_annual(
    cafe_60_atmos, f5_hcst.init[0], dim="time"
)
cafe_60_atmos = utils.round_to_start_of_month(cafe_60_atmos, dim="time")

# Focus on global mean for now
cafe_60 = global_mean(cafe_60_atmos, cafe_atmos_area).mean("ensemble").compute()

### Ocean variables

In [ ]:
data_dir = "/g/data/xv83/dcfp/CAFE60v1/"
realm = "ocean_month.zarr.zip"
variable = {"sst": 1}

cafe_60_ocean = xr.open_dataset(f"{data_dir}{realm}", engine="zarr", chunks={})[
    variable.keys()
]

# Rescale variables
for k, v in variable.items():
    cafe_60_ocean[k] = cafe_60_ocean[k] * v

cafe_60_ocean = utils.coarsen_monthly_to_annual(
    cafe_60_ocean, f5_hcst.init[0], dim="time"
)
cafe_60_ocean = utils.round_to_start_of_month(cafe_60_ocean, dim="time")

# Focus on global mean for now
cafe_60 = xr.merge(
    [
        cafe_60,
        global_mean(cafe_60_ocean, cafe_ocean_area, ["xt_ocean", "yt_ocean"])
        .mean("ensemble")
        .compute(),
    ]
)

# Prepare some data - CAFE historical

### Atmospheric variables

In [ ]:
data_dir = "/g/data/xv83/users/ds0092/data/CAFE/historical/WIP"
realm = "atmos_isobaric_month.zarr.zip"
variable = {"t_ref": 1, "precip": 86400}  # Including scale factor

cafe_hist_atmos = utils.truncate_latitudes(
    xr.open_zarr(f"{data_dir}/c5-d60-pX-hist-19601101/ZARR/{realm}")
)[variable.keys()]
cafe_ctrl_atmos = utils.truncate_latitudes(
    xr.open_zarr(f"{data_dir}/c5-d60-pX-ctrl-19601101/ZARR/{realm}")
)[variable.keys()]

cafe_ctrl_atmos_drift = (
    cafe_ctrl_atmos.mean("ensemble")
    .groupby("time.month")
    .map(lambda x: x - x.mean(["time"]))
)
cafe_hist_atmos = cafe_hist_atmos - cafe_ctrl_atmos_drift

# Rescale variables
for k, v in variable.items():
    cafe_hist_atmos[k] = cafe_hist_atmos[k] * v

cafe_hist_atmos = utils.round_to_start_of_month(cafe_hist_atmos, dim="time")
cafe_hist_atmos = utils.coarsen_monthly_to_annual(cafe_hist_atmos, f5_hcst.init[0])

# Focus on global mean for now
cafe_hist = global_mean(cafe_hist_atmos, cafe_atmos_area).mean("ensemble").compute()

### Ocean variables

In [ ]:
data_dir = "/g/data/xv83/users/ds0092/data/CAFE/historical/WIP"
realm = "ocean_month.zarr.zip"
variable = {"sst": 1}  # Including scale factor

cafe_hist_ocean = utils.truncate_latitudes(
    xr.open_zarr(f"{data_dir}/c5-d60-pX-hist-19601101/ZARR/{realm}")
)[variable.keys()]
cafe_ctrl_ocean = utils.truncate_latitudes(
    xr.open_zarr(f"{data_dir}/c5-d60-pX-ctrl-19601101/ZARR/{realm}")
)[variable.keys()]

cafe_ctrl_ocean_drift = (
    cafe_ctrl_ocean.mean("ensemble")
    .groupby("time.month")
    .map(lambda x: x - x.mean(["time"]))
)
cafe_hist_ocean = cafe_hist_ocean - cafe_ctrl_ocean_drift

# Rescale variables
for k, v in variable.items():
    cafe_hist_ocean[k] = cafe_hist_ocean[k] * v

cafe_hist_ocean = utils.round_to_start_of_month(cafe_hist_ocean, dim="time")
cafe_hist_ocean = utils.coarsen_monthly_to_annual(cafe_hist_ocean, f5_hcst.init[0])

# Focus on global mean for now
cafe_hist = xr.merge(
    [
        cafe_hist,
        global_mean(cafe_hist_ocean, cafe_ocean_area, ["xt_ocean", "yt_ocean"])
        .mean("ensemble")
        .compute(),
    ]
)

# Prepare some data - JRA55

In [ ]:
path = "/g/data/xv83/reanalyses/JRA55/surface_month_cafe-grid.zarr.zip"
variable = ["TMP_GDS0_HTGL", "TPRAT_GDS0_SFC"]

jra55 = utils.truncate_latitudes(
    xr.open_zarr(
        path,
        use_cftime=True,
    )
)[variable]

jra55 = jra55.rename(
    {
        "initial_time0_hours": "time",
        "TMP_GDS0_HTGL": "t_ref",
        "TPRAT_GDS0_SFC": "precip",
    }
)
# Force to Julian calendar
jra55 = jra55.assign_coords(
    {
        "time": xr.cftime_range(
            start=jra55.time[0].item().strftime(),
            end=jra55.time[-1].item().strftime(),
            freq="MS",
            calendar="julian",
        )
    }
)

# Process annual means to match hcst dataset
jra55 = utils.coarsen_monthly_to_annual(jra55, f5_hcst.init[0])

# Focus on global mean for now
obs = global_mean(jra55, cafe_atmos_area).compute()

# Prepare some data - HadISST

In [ ]:
had = xr.open_dataset(
    "/g/data/xv83/reanalyses/HadISST/ocean_month.zarr",
    engine="zarr",
    chunks={},
    use_cftime=True,
)
had = had.where(had > -1000)["sst"].rename({"longitude": "lon", "latitude": "lat"})

had = utils.round_to_start_of_month(had, dim="time")

# Force to Julian calendar
had = had.assign_coords(
    {
        "time": xr.cftime_range(
            start=had.time[0].item().strftime(),
            end=had.time[-1].item().strftime(),
            freq="MS",
            calendar="julian",
        )
    }
)

had = utils.coarsen_monthly_to_annual(had, f5_hcst.init[0])

# Focus on global mean for now
had_area = (
    utils.estimate_cell_areas(had).broadcast_like(had, exclude=["time"]).fillna(0)
)
obs = xr.merge([obs, global_mean(had, had_area).compute()])

# Remove the biases

### Calculate the biases from the f6 climatology over the max consistent period

In [ ]:
def bias_mean(hcst, obs):
    hcst_clim = hcst.mean("init")
    obs_clim = obs.mean("time")
    return hcst_clim - obs_clim

In [ ]:
period = [cftime.DatetimeJulian(1991, 10, 1), cftime.DatetimeJulian(2020, 10, 1)]

bias = bias_mean(
    f6_hcst.where((f6_hcst.time >= period[0]) & (f6_hcst.time <= period[1])),
    obs.where((obs.time >= period[0]) & (obs.time <= period[1])),
)


f6_hcst_bc = f6_hcst - bias

In [ ]:
def plot_hindcasts(hcsts, hcsts_bc):
    from matplotlib.pyplot import cm
    from matplotlib.dates import date2num

    def shading(ax):
        trans = cftime.DatetimeJulian(1992, 1, 1)
        end = cftime.DatetimeJulian(2040, 1, 1)
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.fill_between(
            [trans, end],
            [ylim[1], ylim[1]],
            [ylim[0], ylim[0]],
            color=[0.9, 0.9, 0.9],
        )
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

    n_vars = len(hcsts.data_vars)

    fig = plt.figure(figsize=(10, n_vars * 4))
    axs = fig.subplots(n_vars, 1, sharex=True)
    if n_vars == 1:
        axs = [axs]

    for a, var in enumerate(hcsts.data_vars):
        color = cm.autumn(np.linspace(0, 0.9, len(hcsts[var].init)))
        color_bc = cm.winter(np.linspace(0, 0.9, len(hcsts[var].init)))
        for idx, (i, c, c_bc) in enumerate(zip(hcsts[var].init, color, color_bc)):
            if idx == 0:
                label = "CAFE-f6 raw"
                label_bc = "CAFE-f6 bias corrected"
            else:
                label = label_bc = "_nolabel_"

            hcst = hcsts[var].sel(init=i)
            axs[a].plot(hcst.time[0], hcst[0], color=c, marker="o", label="_nolabel_")
            axs[a].plot(hcst.time, hcst, color=c, linestyle="-", label=label)

            hcst_bc = hcsts_bc[var].sel(init=i)
            axs[a].plot(
                hcst_bc.time[0], hcst_bc[0], color=c_bc, marker="o", label="_nolabel_"
            )
            axs[a].plot(
                hcst_bc.time, hcst_bc, color=c_bc, linestyle="-", label=label_bc
            )

        axs[a].plot(obs.time, obs[var], color="black", label="Observations")
        axs[a].plot(
            cafe_60.time, cafe_60[var], color="black", linestyle="--", label="CAFE60v1"
        )
        axs[a].plot(
            cafe_hist.time, cafe_hist[var], color="grey", label="CAFE-historical"
        )

        ticks = xr.cftime_range(
            start="1981-01-01", end="2032-01-01", freq="2AS", calendar="julian"
        )
        axs[a].set_xticks(ticks.values)
        axs[a].set_xticklabels(ticks.year)
        axs[a].set_xlim(
            cftime.DatetimeJulian(1981, 1, 1), cftime.DatetimeJulian(2032, 1, 1)
        )
        axs[a].set_ylabel(hcsts[var].attrs["long_name"])
        axs[a].grid()
        if a == 0:
            axs[a].legend()
        if a == (n_vars - 1):
            axs[a].set_xlabel("year")
        else:
            axs[a].set_xlabel("")

        shading(axs[a])

    plt.tight_layout()

    return fig

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### That obviously has quite a large step around 1992 due to the change to CAFE60's bias-correction scheme.

### What about if we calculate anomalies for pre and post 1992 separately? Let's use CAFE-f5 so that we can use the same length period for bias correction pre and post the change.

In [ ]:
transition_year = 1992

period_1 = [
    cftime.DatetimeJulian(1973, 10, 1),
    cftime.DatetimeJulian(1992, 10, 1),
]  # 20 years, using initial dates < 1992
period_2 = [
    cftime.DatetimeJulian(2002, 10, 1),
    cftime.DatetimeJulian(2021, 10, 1),
]  # 20 years, using initial dates >= 1992

bias_1 = bias_mean(
    f5_hcst.where((f5_hcst.time >= period_1[0]) & (f5_hcst.time <= period_1[1])),
    obs.where((obs.time >= period_1[0]) & (obs.time <= period_1[1])),
)

bias_2 = bias_mean(
    f5_hcst.where((f5_hcst.time >= period_2[0]) & (f5_hcst.time <= period_2[1])),
    obs.where((obs.time >= period_2[0]) & (obs.time <= period_2[1])),
)

f6_hcst_bc1 = f6_hcst.sel(init=slice(None, str(transition_year - 1))) - bias_1
f6_hcst_bc2 = f6_hcst.sel(init=slice(str(transition_year), None)) - bias_2
f6_hcst_bc = xr.concat((f6_hcst_bc1, f6_hcst_bc2), dim="init")

In [ ]:
fig, ax = plt.subplots(figsize=(4, 8))

x = np.arange(len(f5_hcst.lead) + 1)
y = np.arange(1, len(f5_hcst.init) + 2) - 0.5
c = f5_hcst["sst"].transpose("init", "lead")
im = ax.pcolor(x, y, c)

ax.set_xticks(np.arange(len(f5_hcst.lead)) + 0.5)
ax.set_xticklabels(f5_hcst.lead.values)
ax.set_yticks(np.arange(1, len(f5_hcst.init) + 1, 2))
ax.set_yticklabels(f5_hcst.init.dt.year.values[::2])


def hatch(mask, color):
    import matplotlib.patches as mpatches

    for j, i in np.column_stack(np.where(mask)):
        ax.add_patch(
            mpatches.Rectangle(
                (i, j + 0.5),
                1,
                1,
                fill=False,
                linewidth=0,
                snap=False,
                color=color,
                hatch="///",
            )
        )


mask = xr.ones_like(f5_hcst["t_ref"]).where(
    (f5_hcst.time >= period_1[0]) & (f5_hcst.time <= period_1[1]),
    0,
)
hatch(mask, "c")
mask = xr.ones_like(f5_hcst["t_ref"]).where(
    (f5_hcst.time >= period_2[0]) & (f5_hcst.time <= period_2[1]),
    0,
)
hatch(mask, "m")

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### Looks okay for `t_ref` and `sst` but not `precip`... What about if we do the same thing but don't limit ourselves to a consistent period at each lead

In [ ]:
transition_year = 1992

period_1 = slice(
    cftime.DatetimeJulian(1963, 11, 1),
    cftime.DatetimeJulian(1991, 11, 1),
)  # 28 years
period_2 = slice(
    cftime.DatetimeJulian(1992, 11, 1),
    cftime.DatetimeJulian(2020, 11, 1),
)  # 28 years

f5_period_1 = f5_hcst.sel(init=period_1)
bias_1 = []
for l in f5_period_1.lead:
    ts = f5_period_1.sel(lead=l).swap_dims({"init": "time"})
    bias_1.append((ts - obs).mean("time").expand_dims({"lead": [l]}))
bias_1 = xr.concat(bias_1, dim="lead")

f5_period_2 = f5_hcst.sel(init=period_2)
bias_2 = []
for l in f5_period_2.lead:
    ts = f5_period_2.sel(lead=l).swap_dims({"init": "time"})
    bias_2.append((ts - obs).mean("time").expand_dims({"lead": [l]}))
bias_2 = xr.concat(bias_2, dim="lead")

f6_hcst_bc1 = f6_hcst.sel(init=slice(None, str(transition_year - 1))) - bias_1
f6_hcst_bc2 = f6_hcst.sel(init=slice(str(transition_year), None)) - bias_2
f6_hcst_bc = xr.concat((f6_hcst_bc1, f6_hcst_bc2), dim="init")

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### Still not great... What about correcting for the biases on a sliding window basis, using the previous 12 years of data?

e.g. 
- for the 1981 forecast, biases are calculated over the period 1970-1981 at all leads
- for the 2020 forecast, biases are calculated over the period 2009-2020 at all leads 

This is a fair approach, but it will have to use the f5 forecasts to calculate the biases

In [ ]:
n_years = 12

objs = []
for init in f6_hcst.init:
    hcst = f6_hcst.sel(init=[init.item()])

    # Calculate the bias
    clim_range = [
        hcst.get_index("init").shift(-(12 * n_years), "MS").item(),
        hcst.get_index("init").item(),
    ]
    bias = bias_mean(
        f5_hcst.where(
            (f5_hcst.time >= clim_range[0]) & (f5_hcst.time <= clim_range[1])
        ),
        obs.where((obs.time >= clim_range[0]) & (obs.time <= clim_range[1])),
    )

    objs.append(hcst - bias)

f6_hcst_bc = xr.concat(objs, dim="init")

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### What about doing this in an unfair way, selecting the period based on initial date alone?
This approach uses observations from the future to perform the bias correction at non-zero leads

In [ ]:
n_years = 20

objs = []
for init in f6_hcst.init:
    hcst = f6_hcst.sel(init=[init.item()])

    init_range = slice(
        hcst.get_index("init").shift(-(12 * (n_years - 1)), "MS").item(),
        hcst.get_index("init").item(),
    )
    f5_period = f5_hcst.sel(init=init_range)
    bias = []
    for l in f5_period.lead:
        ts = f5_period.sel(lead=l).swap_dims({"init": "time"})
        bias.append((ts - obs).mean("time").expand_dims({"lead": [l]}))
    bias = xr.concat(bias, dim="lead")

    objs.append(hcst - bias)

f6_hcst_bc = xr.concat(objs, dim="init")

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### What about fitting an exponential?
Of the form $D(l>0) = D(l=0) \exp(-A l)$ where $D = F - H$ is the deviation between the forecast, $F$, and the historical run, $H$; and $l$ is the lead time.

In [ ]:
var = "t_ref"  # Only works on DataArrays

In [ ]:
res = []
for i in f6_hcst.init:
    hcst = f6_hcst.sel(init=i).swap_dims({"lead": "time"})
    res.append((hcst - cafe_hist)[var].swap_dims({"time": "lead"}))

f6_D = xr.concat(res, dim="init")

In [ ]:
from scipy.optimize import leastsq


def multiple_reg(x, y, f, const, params0, **kwargs):
    """Do same non-linear regression on multiple curves"""

    def leastsq_func(params, *args):
        x, y = args[:2]
        const = args[2:]
        yfit = []
        for i in range(len(x)):
            yfit = np.append(yfit, f(x[i], *const[i], *params))
        return y - yfit

    # turn const into 2d-array if 1d is given
    const = np.asarray(const)
    if len(const.shape) < 2:
        const = np.atleast_2d(const).T

    # ensure that y is flat and x is nested
    if hasattr(y[0], "__len__"):
        y = [item for sublist in y for item in sublist]
    if not hasattr(x[0], "__len__"):
        x = np.tile(x, (len(const), 1))
    x_ = [item for sublist in x for item in sublist]
    assert len(x_) == len(y)

    # collect all arguments in a tuple
    y = np.asarray(y)
    args = [x, y] + list(const)
    args = tuple(args)  # doesn't work if args is a list!!

    return leastsq(leastsq_func, params0, args=args, **kwargs)

In [ ]:
def fit(x, D0, A, B):
    return B * D0 * np.exp(-A * x)  # B * D0 + B * D0 * np.tanh(-A * x)  #


p, _ = multiple_reg(
    f6_D.lead.values[1:], f6_D.values[:, 1:], fit, f6_D.values[:, 0], [0.1, 1]
)

In [ ]:
f6_hcst_bc = []
f6_hcst_fit = []
for i in f6_D.init:
    f6_D_ts = f6_D.sel(init=i).swap_dims({"lead": "time"})
    f6_hcst_ts = f6_hcst.sel(init=i).swap_dims({"lead": "time"})
    D_fit = fit(f6_D_ts.lead, f6_D_ts.values[0], p[0], p[1])
    D_fit = xr.zeros_like(f6_hcst_ts) + D_fit
    f6_hcst_bc.append((f6_hcst_ts - D_fit).swap_dims({"time": "lead"}))
    f6_hcst_fit.append((D_fit + cafe_hist).swap_dims({"time": "lead"}))

f6_hcst_bc = xr.concat(f6_hcst_bc, dim="init")
f6_hcst_fit = xr.concat(f6_hcst_fit, dim="init")

period = [cftime.DatetimeJulian(1991, 10, 1), cftime.DatetimeJulian(2020, 10, 1)]

f6_clim = f6_hcst_bc.where(
    (f6_hcst_bc.time >= period[0]) & (f6_hcst_bc.time <= period[1])
).mean("init")
jra_clim = obs.where((obs.time >= period[0]) & (obs.time <= period[1])).mean("time")
bias = (f6_clim - jra_clim).mean("lead")

f6_hcst_bc = f6_hcst_bc - bias

In [ ]:
_ = plot_hindcasts(f6_hcst[[var]], f6_hcst_bc[[var]])

for i in f6_hcst_fit.init:
    plot_fit = f6_hcst_fit[var].sel(init=i).swap_dims({"lead": "time"})
    plt.plot(plot_fit.time, plot_fit, color="k", linestyle=":")

### Perhaps there is a clever way to utilize the historical run in the bias correction, but this is not it... 

### What about splitting about 1992 and using the Kharin approach?
Perhaps this will make the transition a little smoother, though fitting a trend over 20 years is questionable...

In [ ]:
def bias_Kharin(hcst, obs):
    def _linear_fit_in_time(da):
        p = []
        for l in da.lead:
            ts = da.sel(lead=l).swap_dims({"init": "time"})
            p.append(ts.polyfit(dim="time", deg=1).expand_dims(lead=[l]))
        return xr.concat(p, dim="lead")

    hcst_coeffs = _linear_fit_in_time(hcst)
    hcst_coeffs = hcst_coeffs.rename(
        {n: n.removesuffix("_polyfit_coefficients") for n in hcst_coeffs.data_vars}
    )
    obs_coeffs = obs.polyfit("time", deg=1)
    obs_coeffs = obs_coeffs.rename(
        {n: n.removesuffix("_polyfit_coefficients") for n in obs_coeffs.data_vars}
    )

    slope_bias_coeffs = hcst_coeffs - obs_coeffs
    slope_bias = xr.polyval(hcst["init"], slope_bias_coeffs)
    hcst_slope_corrected = hcst - slope_bias

    bias = bias_mean(hcst_slope_corrected, obs)

    return slope_bias + bias

In [ ]:
transition_year = 1992

period_1 = [
    cftime.DatetimeJulian(1973, 10, 1),
    cftime.DatetimeJulian(1992, 10, 1),
]  # 20 years, using initial dates < 1992
period_2 = [
    cftime.DatetimeJulian(2002, 10, 1),
    cftime.DatetimeJulian(2021, 10, 1),
]  # 20 years, using initial dates >= 1992

bias_1 = bias_Kharin(
    f5_hcst.where((f5_hcst.time >= period_1[0]) & (f5_hcst.time <= period_1[1])),
    obs.where((obs.time >= period_1[0]) & (obs.time <= period_1[1])),
)

bias_2 = bias_Kharin(
    f5_hcst.where((f5_hcst.time >= period_2[0]) & (f5_hcst.time <= period_2[1])),
    obs.where((obs.time >= period_2[0]) & (obs.time <= period_2[1])),
)

f6_hcst_bc1 = f6_hcst.sel(init=slice(None, str(transition_year - 1))) - bias_1
f6_hcst_bc2 = f6_hcst.sel(init=slice(str(transition_year), None)) - bias_2
f6_hcst_bc = xr.concat((f6_hcst_bc1, f6_hcst_bc2), dim="init")

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### This introduces a discontinuity. What about first removing the mean bias either side of 1992 first, then adjusting the slope?

In [ ]:
transition_year = 1992

period_1 = [
    cftime.DatetimeJulian(1973, 10, 1),
    cftime.DatetimeJulian(1992, 10, 1),
]  # 20 years, using initial dates < 1992
period_2 = [
    cftime.DatetimeJulian(2002, 10, 1),
    cftime.DatetimeJulian(2021, 10, 1),
]  # 20 years, using initial dates >= 1992

bias_1 = bias_mean(
    f5_hcst.where((f5_hcst.time >= period_1[0]) & (f5_hcst.time <= period_1[1])),
    obs.where((obs.time >= period_1[0]) & (obs.time <= period_1[1])),
)

bias_2 = bias_mean(
    f5_hcst.where((f5_hcst.time >= period_2[0]) & (f5_hcst.time <= period_2[1])),
    obs.where((obs.time >= period_2[0]) & (obs.time <= period_2[1])),
)

f6_hcst_bc1 = f6_hcst.sel(init=slice(None, str(transition_year - 1))) - bias_1
f6_hcst_bc2 = f6_hcst.sel(init=slice(str(transition_year), None)) - bias_2
f6_hcst_bc = xr.concat((f6_hcst_bc1, f6_hcst_bc2), dim="init")


# Adjust the slope
period = [cftime.DatetimeJulian(1991, 10, 1), cftime.DatetimeJulian(2020, 10, 1)]

bias = bias_Kharin(
    f6_hcst_bc.where((f6_hcst.time >= period[0]) & (f6_hcst.time <= period[1])),
    obs.where((obs.time >= period[0]) & (obs.time <= period[1])),
)

f6_hcst_bc = f6_hcst_bc - bias

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### The inclusion of the slope adjustment does not obviously improve things. What about just using the Kharin approach outright. This feels a little "off" because it's using a feature of the approach for the wrong reason...

In [ ]:
period = [
    cftime.DatetimeJulian(1971, 10, 1),
    cftime.DatetimeJulian(2020, 10, 1),
]

bias = bias_Kharin(
    f5_hcst.where((f5_hcst.time >= period[0]) & (f5_hcst.time <= period[1])),
    obs.where((obs.time >= period[0]) & (obs.time <= period[1])),
)

f6_hcst_bc = f6_hcst - bias

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### Last thing to try: remove an offset calculated from CAFE-60, prior to performing the bias correction

In [ ]:
# Subtract the offset
transition_year = 1992
period_1 = slice("1964", str(transition_year - 1))  # 28 years
period_2 = slice(str(transition_year), "2019")  # 28 years

bias_1 = cafe_60.sel(time=period_1).mean("time") - obs.sel(time=period_1).mean("time")
bias_2 = cafe_60.sel(time=period_2).mean("time") - obs.sel(time=period_2).mean("time")

f6_hcst_bc1 = f6_hcst.sel(init=slice(None, str(transition_year - 1))) - bias_1
f6_hcst_bc2 = f6_hcst.sel(init=slice(str(transition_year), None)) - bias_2
f6_hcst_bc = xr.concat((f6_hcst_bc1, f6_hcst_bc2), dim="init")

# Lead dependent bias correction
period = [cftime.DatetimeJulian(1991, 10, 1), cftime.DatetimeJulian(2020, 10, 1)]

bias = bias_mean(
    f6_hcst_bc.where((f6_hcst_bc.time >= period[0]) & (f6_hcst_bc.time <= period[1])),
    obs.where((obs.time >= period[0]) & (obs.time <= period[1])),
)

f6_hcst_bc = f6_hcst_bc - bias

In [ ]:
_ = plot_hindcasts(f6_hcst, f6_hcst_bc)

### Could we just do all verification relative to CAFE60v1? How would we interpret this? Can we justify that this is a sensible thing to do?

In [ ]:
cluster.close()
client.close()